In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv("../EDA/dataframe_model/stretchedSociety_block_109_MAC005547.csv")

In [18]:
df.head()

,time,Energy_kwh,house_hold,temperature,windSpeed,precipType,icon,summary,holiday,bool_weather_missing_values,year,month,day,hour,dayofweek_num
0,2013-01-01 00:00:00,0.497,0,7.01,5.46,0,0,0,1,0,0,0,0,0,0
1,2013-01-01 01:00:00,0.314,0,7.49,5.51,0,0,0,0,0,0,0,0,1,0
2,2013-01-01 02:00:00,0.202,0,7.16,5.74,0,0,0,0,0,0,0,0,2,0
3,2013-01-01 03:00:00,0.171,0,7.04,4.98,0,0,0,0,0,0,0,0,3,0
4,2013-01-01 04:00:00,0.185,0,6.29,4.20,0,0,0,0,0,0,0,0,4,0


In [19]:
df['time'] = pd.to_datetime(df['time'])

In [20]:
df[['time','Energy_kwh','holiday','month','hour','dayofweek_num']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10153 entries, 0 to 10152
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   time           10153 non-null  datetime64[ns]
 1   Energy_kwh     10153 non-null  float64       
 2   holiday        10153 non-null  int64         
 3   month          10153 non-null  int64         
 4   hour           10153 non-null  int64         
 5   dayofweek_num  10153 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 476.0 KB


In [21]:
df = df[['time','Energy_kwh','holiday','month','hour','dayofweek_num']]

In [22]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

# Função para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

# Função para calcular SMAPE
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Preparar os dados
df_prophet = df.rename(columns={'time': 'ds', 'Energy_kwh': 'y'})

# Dividir os dados em treino e teste
train_size = len(df) - 168
df_train = df_prophet[:train_size]
df_test = df_prophet[train_size:]

# Criar e treinar o modelo Prophet
start_time = time.time()

model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
model.add_regressor('holiday')
model.add_regressor('month')
model.add_regressor('hour')
model.add_regressor('dayofweek_num')

model.fit(df_train)

training_time = time.time() - start_time

# Fazer previsões
future = df_test[['ds', 'holiday', 'month', 'hour', 'dayofweek_num']]
forecast = model.predict(future)

# Calcular métricas
y_true = df_test['y'].values
y_pred = forecast['yhat'].values

mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
smape = symmetric_mean_absolute_percentage_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Salvar resultados em CSV
results = pd.DataFrame({
    'model': ['Prophet'],
    'mae': [mae],
    'mape': [mape],
    'smape': [smape],
    'rmse': [rmse],
    'training_time': [training_time]
})

results.to_csv('prophet_results.csv', index=False)

print("Resultados salvos em 'prophet_results.csv'")
print(results)

08:26:14 - cmdstanpy - INFO - Chain [1] start processing
08:26:14 - cmdstanpy - INFO - Chain [1] done processing


Resultados salvos em 'prophet_results.csv'
     model       mae      mape     smape      rmse  training_time
0  Prophet  0.117594  0.464684  0.366027  0.152383       0.819523


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

# Função para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Função para calcular SMAPE
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Preparar os dados
df_sarimax = df.set_index('time')

# Dividir os dados em treino e teste
train_size = len(df) - 168
df_train = df_sarimax[:train_size]
df_test = df_sarimax[train_size:]

# Definir variáveis exógenas
exog_vars = ['holiday', 'month', 'hour', 'dayofweek_num']

# Criar e treinar o modelo SARIMAX
start_time = time.time()

# Aqui, estamos usando um modelo SARIMAX(1,1,1)(1,1,1,24) como exemplo
# Você pode ajustar esses parâmetros conforme necessário
model = SARIMAX(df_train['Energy_kwh'], 
                exog=df_train[exog_vars],
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 24),
                enforce_stationarity=False,
                enforce_invertibility=False)

results = model.fit()

training_time = time.time() - start_time

# Fazer previsões
forecast = results.get_forecast(steps=168, exog=df_test[exog_vars])
y_pred = forecast.predicted_mean

# Calcular métricas
y_true = df_test['Energy_kwh'].values
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
smape = symmetric_mean_absolute_percentage_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Salvar resultados em CSV
results_df = pd.DataFrame({
    'model': ['SARIMAX'],
    'mae': [mae],
    'mape': [mape],
    'smape': [smape],
    'rmse': [rmse],
    'training_time': [training_time]
})

results_df.to_csv('sarimax_results.csv', index=False)

print("Resultados salvos em 'sarimax_results.csv'")
print(results_df)


In [4]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

# Assumindo que df já está definido e contém os dados necessários

# Passo 1: Preparação dos dados
print("Passo 1: Preparação dos dados")
df_model = df[['time', 'Energy_kwh', 'holiday', 'month', 'hour', 'dayofweek_num']].copy()
df_model = df_model.rename(columns={'time': 'ds', 'Energy_kwh': 'y'})
print(df_model.head())
print(f"Shape dos dados: {df_model.shape}")

# Passo 2: Definir o ponto de divisão para treinamento/teste
print("\nPasso 2: Divisão dos dados")
split_point = len(df_model) - 168
df_train = df_model[:split_point]
df_test = df_model[split_point:]
print(f"Shape do conjunto de treino: {df_train.shape}")
print(f"Shape do conjunto de teste: {df_test.shape}")

# Passo 3: Configuração do modelo Neural Prophet
print("\nPasso 3: Configuração do modelo Neural Prophet")
model = NeuralProphet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    batch_size=64,
    epochs=100,
)

# Adicionar as covariáveis
for regressor in ['holiday', 'month', 'hour', 'dayofweek_num']:
    model.add_future_regressor(regressor)

# Passo 4: Treinamento do modelo e medição do tempo
print("\nPasso 4: Treinamento do modelo")
start_time = time.time()
metrics = model.fit(df_train, freq='H')
training_time = time.time() - start_time
print(f"Tempo de treinamento: {training_time:.2f} segundos")

# Passo 5: Fazer previsões
print("\nPasso 5: Fazendo previsões")
# Usamos df_test diretamente, pois já contém os valores futuros dos regressores
forecast = model.predict(df_test)
print(forecast.head())

# Passo 6: Calcular métricas
print("\nPasso 6: Cálculo das métricas")
y_true = df_test['y'].values
y_pred = forecast['yhat1'].values

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

# Passo 7: Salvar resultados em CSV
print("\nPasso 7: Salvando resultados")
results = pd.DataFrame({
    'Model': ['NeuralProphet'],
    'MAE': [mae],
    'RMSE': [rmse],
    'MAPE': [mape],
    'SMAPE': [smape],
    'Training_Time': [training_time]
})

results.to_csv('neural_prophet_results.csv', index=False)

print("\nResultados salvos em 'neural_prophet_results.csv'")
print(results)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.99% of the data.
WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1173: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  aux_ts = pd.DataFrame(pd.date_range("1994-01-01", periods=100, freq=freq_str))

WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packag

Passo 1: Preparação dos dados
                    ds      y  holiday  month  hour  dayofweek_num
0  2013-01-01 00:00:00  0.497        1      0     0              0
1  2013-01-01 01:00:00  0.314        0      0     1              0
2  2013-01-01 02:00:00  0.202        0      0     2              0
3  2013-01-01 03:00:00  0.171        0      0     3              0
4  2013-01-01 04:00:00  0.185        0      0     4              0
Shape dos dados: (10153, 6)

Passo 2: Divisão dos dados
Shape do conjunto de treino: (9985, 6)
Shape do conjunto de teste: (168, 6)

Passo 3: Configuração do modelo Neural Prophet

Passo 4: Treinamento do modelo


Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (157) is too small than the required number                     for the learning rate finder (250). The results might not be optimal.


Finding best initial lr:   0%|          | 0/250 [00:00<?, ?it/s]

...

WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.405% of the data.
WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1173: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  aux_ts = pd.DataFrame(pd.date_range("1994-01-01", periods=100, freq=freq_str))

WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use 

Tempo de treinamento: 129.06 segundos

Passo 5: Fazendo previsões


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


                   ds      y     yhat1     trend  season_yearly  \
0 2014-02-21 01:00:00  0.555  0.307385  0.334974       0.059196   
1 2014-02-21 02:00:00  0.588  0.226512  0.334983       0.059258   
2 2014-02-21 03:00:00  0.398  0.195170  0.334993       0.059319   
3 2014-02-21 04:00:00  0.163  0.183428  0.335002       0.059380   
4 2014-02-21 05:00:00  0.159  0.181712  0.335012       0.059443   

   season_weekly  season_daily  future_regressors_additive  \
0      -0.020100     -0.065280                   -0.001405   
1      -0.019357     -0.146040                   -0.002332   
2      -0.018531     -0.177351                   -0.003258   
3      -0.017640     -0.189129                   -0.004185   
4      -0.016663     -0.190968                   -0.005112   

   future_regressor_dayofweek_num  future_regressor_holiday  \
0                       -0.005474                       0.0   
1                       -0.005474                       0.0   
2                       -0.005474  

In [5]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

# Assumindo que df já está definido e contém os dados necessários

# Passo 1: Preparação dos dados
print("Passo 1: Preparação dos dados")
df_model = df[['time', 'Energy_kwh', 'holiday', 'month', 'hour', 'dayofweek_num']].copy()
df_model = df_model.rename(columns={'time': 'ds', 'Energy_kwh': 'y'})
print(df_model.head())
print(f"Shape dos dados: {df_model.shape}")

# Passo 2: Configuração do modelo Neural Prophet
print("\nPasso 2: Configuração do modelo Neural Prophet")
model = NeuralProphet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    batch_size=64,
    epochs=100,
)

# Adicionar as covariáveis
for regressor in ['holiday', 'month', 'hour', 'dayofweek_num']:
    model.add_future_regressor(regressor)

# Passo 3: Divisão dos dados usando split_df
print("\nPasso 3: Divisão dos dados")
df_train, df_test = model.split_df(df_model, valid_p=168/len(df_model))
print(f"Shape do conjunto de treino: {df_train.shape}")
print(f"Shape do conjunto de teste: {df_test.shape}")

# Passo 4: Treinamento do modelo e medição do tempo
print("\nPasso 4: Treinamento do modelo")
start_time = time.time()
metrics = model.fit(df_train, freq='H')
training_time = time.time() - start_time
print(f"Tempo de treinamento: {training_time:.2f} segundos")

# Passo 5: Criar dataframe futuro e fazer previsões
print("\nPasso 5: Criando dataframe futuro e fazendo previsões")
future_df = model.make_future_dataframe(df_train, periods=168, regressors_df=df_test)
forecast = model.predict(future_df)
print(forecast.head())

# Passo 6: Calcular métricas
print("\nPasso 6: Cálculo das métricas")
y_true = df_test['y'].values
y_pred = forecast['yhat1'].values[:168]  # Pegamos apenas as primeiras 168 previsões

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

# Passo 7: Salvar resultados em CSV
print("\nPasso 7: Salvando resultados")
results = pd.DataFrame({
    'Model': ['NeuralProphet'],
    'MAE': [mae],
    'RMSE': [rmse],
    'MAPE': [mape],
    'SMAPE': [smape],
    'Training_Time': [training_time]
})

results.to_csv('neural_prophet_results.csv', index=False)

print("\nResultados salvos em 'neural_prophet_results.csv'")
print(results)

WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.99% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as h
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated 

Passo 1: Preparação dos dados
                    ds      y  holiday  month  hour  dayofweek_num
0  2013-01-01 00:00:00  0.497        1      0     0              0
1  2013-01-01 01:00:00  0.314        0      0     1              0
2  2013-01-01 02:00:00  0.202        0      0     2              0
3  2013-01-01 03:00:00  0.171        0      0     3              0
4  2013-01-01 04:00:00  0.185        0      0     4              0
Shape dos dados: (10153, 6)

Passo 2: Configuração do modelo Neural Prophet

Passo 3: Divisão dos dados
Shape do conjunto de treino: (9985, 6)
Shape do conjunto de teste: (168, 6)

Passo 4: Treinamento do modelo


Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (157) is too small than the required number                     for the learning rate finder (250). The results might not be optimal.


Finding best initial lr:   0%|          | 0/250 [00:00<?, ?it/s]

...

Tempo de treinamento: 125.41 segundos

Passo 5: Criando dataframe futuro e fazendo previsões


WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.99% of the data.
WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1173: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  aux_ts = pd.DataFrame(pd.date_range("1994-01-01", periods=100, freq=freq_str))

WARNING - (py.warnings._showwarnmsg) - /home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use `

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


                   ds     y     yhat1     trend  season_yearly  season_weekly  \
0 2014-02-21 01:00:00  None  0.306050  0.385866       0.006615      -0.021134   
1 2014-02-21 02:00:00  None  0.223760  0.385886       0.006683      -0.020446   
2 2014-02-21 03:00:00  None  0.192097  0.385906       0.006749      -0.019680   
3 2014-02-21 04:00:00  None  0.180905  0.385926       0.006817      -0.018834   
4 2014-02-21 05:00:00  None  0.178642  0.385945       0.006884      -0.017909   

   season_daily  future_regressors_additive  future_regressor_dayofweek_num  \
0     -0.063405                   -0.001891                       -0.005868   
1     -0.145585                   -0.002777                       -0.005868   
2     -0.177214                   -0.003664                       -0.005868   
3     -0.188453                   -0.004550                       -0.005868   
4     -0.190843                   -0.005436                       -0.005868   

   future_regressor_holiday  future_re

In [4]:
csv_list = [
    'establishedAffluence_block_10_MAC001221.csv',
    'establishedAffluence_block_10_MAC001402.csv',
    'establishedAffluence_block_10_MAC001869.csv',
    'establishedAffluence_block_10_MAC002036.csv',
    'establishedAffluence_block_11_MAC001980.csv',
    'establishedAffluence_block_12_MAC002752.csv',
    'establishedAffluence_block_13_MAC005411.csv',
    'establishedAffluence_block_14_MAC002635.csv',
    'establishedAffluence_block_17_MAC002007.csv',
    'establishedAffluence_block_18_MAC005491.csv',
    'establishedAffluence_block_19_MAC002250.csv',
    'establishedAffluence_block_19_MAC005426.csv',
    'establishedAffluence_block_19_MAC005566.csv',
    'establishedAffluence_block_21_MAC004390.csv',
    'establishedAffluence_block_21_MAC004531.csv',
    'establishedAffluence_block_22_MAC004694.csv',
    'establishedAffluence_block_22_MAC004736.csv',
    'establishedAffluence_block_23_MAC000164.csv',
    'establishedAffluence_block_23_MAC004618.csv',
    'establishedAffluence_block_23_MAC004625.csv',
    'establishedAffluence_block_25_MAC004233.csv',
    'establishedAffluence_block_26_MAC002966.csv',
    'establishedAffluence_block_26_MAC003075.csv',
    'establishedAffluence_block_27_MAC003016.csv',
    'establishedAffluence_block_27_MAC005235.csv',
    'establishedAffluence_block_28_MAC004895.csv',
    'establishedAffluence_block_28_MAC005217.csv',
    'establishedAffluence_block_30_MAC000274.csv',
    'establishedAffluence_block_30_MAC003114.csv',
    'establishedAffluence_block_31_MAC001378.csv',
    'establishedAffluence_block_32_MAC001433.csv',
    'establishedAffluence_block_32_MAC002686.csv',
    'establishedAffluence_block_33_MAC000351.csv',
    'establishedAffluence_block_33_MAC002099.csv',
    'establishedAffluence_block_33_MAC004926.csv',
    'establishedAffluence_block_33_MAC004965.csv',
    'establishedAffluence_block_34_MAC001452.csv',
    'establishedAffluence_block_34_MAC002265.csv',
    'establishedAffluence_block_35_MAC001111.csv',
    'establishedAffluence_block_37_MAC000782.csv',
    'establishedAffluence_block_37_MAC000861.csv',
    'establishedAffluence_block_39_MAC001651.csv',
    'establishedAffluence_block_39_MAC001905.csv',
    'establishedAffluence_block_40_MAC000459.csv',
    'establishedAffluence_block_40_MAC005359.csv',
    'stretchedSociety_block_100_MAC002775.csv',
    'stretchedSociety_block_100_MAC002824.csv',
    'stretchedSociety_block_100_MAC002856.csv',
    'stretchedSociety_block_100_MAC002898.csv',
    'stretchedSociety_block_100_MAC003020.csv',
    'stretchedSociety_block_102_MAC003964.csv',
    'stretchedSociety_block_102_MAC003980.csv',
    'stretchedSociety_block_102_MAC003996.csv',
    'stretchedSociety_block_102_MAC004129.csv',
    'stretchedSociety_block_104_MAC000090.csv',
    'stretchedSociety_block_104_MAC004395.csv',
    'stretchedSociety_block_104_MAC004481.csv',
    'stretchedSociety_block_105_MAC002054.csv',
    'stretchedSociety_block_106_MAC000077.csv',
    'stretchedSociety_block_106_MAC000653.csv',
    'stretchedSociety_block_106_MAC001460.csv',
    'stretchedSociety_block_106_MAC002318.csv',
    'stretchedSociety_block_107_MAC001672.csv',
    'stretchedSociety_block_107_MAC001673.csv',
    'stretchedSociety_block_107_MAC001785.csv',
    'stretchedSociety_block_109_MAC000529.csv',
    'stretchedSociety_block_109_MAC002196.csv',
    'stretchedSociety_block_109_MAC002293.csv',
    'stretchedSociety_block_109_MAC005547.csv',
    'stretchedSociety_block_110_MAC002175.csv',
    'stretchedSociety_block_91_MAC000350.csv',
    'stretchedSociety_block_93_MAC000449.csv',
    'stretchedSociety_block_93_MAC004547.csv',
    'stretchedSociety_block_94_MAC000838.csv',
    'stretchedSociety_block_94_MAC000870.csv',
    'stretchedSociety_block_94_MAC004366.csv',
    'stretchedSociety_block_95_MAC000006.csv',
    'stretchedSociety_block_95_MAC000365.csv',
    'stretchedSociety_block_95_MAC004066.csv',
    'stretchedSociety_block_96_MAC001188.csv',
    'stretchedSociety_block_96_MAC001243.csv',
    'stretchedSociety_block_97_MAC000696.csv',
    'stretchedSociety_block_97_MAC001035.csv',
    'stretchedSociety_block_97_MAC001041.csv',
    'stretchedSociety_block_98_MAC001062.csv',
    'stretchedSociety_block_98_MAC004064.csv',
    'stretchedSociety_block_98_MAC004892.csv',
    'stretchedSociety_block_98_MAC004934.csv',
    'stretchedSociety_block_99_MAC004745.csv',
    'stretchedSociety_block_99_MAC004799.csv'
]